In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Creating the Dataframe

In [127]:
df = pd.read_csv('/home/hector/Dropbox/courses/data-sci/08-App-ds-caps/toronto.csv')

In [128]:
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue,Humber Valley Village"
9,M1B,Scarborough,"Malvern,Rouge"


# First Dataframe:

### Removing the "Not assigned" rows in the Borough column

In [130]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue,Humber Valley Village"
9,M1B,Scarborough,"Malvern,Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill,Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District,Ryerson"


### Sorting the date frame in desceding order using the Postal Code

In [207]:
df.sort_values(by = ['Postal Code'], inplace =True)
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Importing the Latitude and Longitud data, and sorting it by Postal code

In [146]:
df_g = pd.read_csv('/home/hector/Dropbox/courses/data-sci/08-App-ds-caps/Geospatial_Coordinates.csv')

In [148]:
df_g.sort_values(by = ['Postal Code'], inplace =True)
df_g.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Lets check if both data frames have the same lenght

In [152]:
print("The lenght for the Toronto dataframe:", df.shape)
print("The length for the geo-data dataframe:", df_g.shape)

The lenght for the Toronto dataframe: (103, 5)
The length for the geo-data dataframe: (103, 3)


In [136]:
df.head(10)

,Postal Code,Borough,Neighbourhood
9,M1B,Scarborough,"Malvern,Rouge"
18,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
27,M1E,Scarborough,"Guildwood,Morningside,West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
54,M1J,Scarborough,Scarborough Village
63,M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park"
72,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge"
81,M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West"
90,M1N,Scarborough,"Birch Cliff,Cliffside West"


# Dataframe with the Latituder and Longitud columns:

Now, that we saw that the length of both dataframe is the same, we can merge them into the final dataframe.
First we will reset the index from df, then we will add the Latitude and Longitude colums 

In [137]:
df.reset_index(drop=True, inplace = True)

In [ ]:
df['Latitude'] = df_g['Latitude']

In [143]:
df['Longitude'] = df_g['Longitude']

In [154]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## Now, locating all the Neighbourhood in Toronto

In [204]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [171]:
df.groupby(['Borough']).count()

,Postal Code,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


In [205]:
#create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df.iloc[:,2]):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Finaly, finding all the Neighbourhoods with "toronto" in the Borough's name

In [202]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
#toronto_data

In [203]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data.iloc[:,2]):
    #print(toronto_data.iloc[2])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor